In [13]:
#Each rule could have one or more arguments
class argument:
    def __init__(self,varname,symbol,varval): #For example if age>45 has varname="age",symbol=">",varval="45"
        self.varname=varname
        self.symbol=symbol
        self.varval=varval   #must have str here because if the values are integers error will occur

In [35]:
#Rule class
class rule:
    def __init__(self,lst,resultvarname,resultvarvalue,accuracy,coverage):     
        #lst:lst of argument classes
        #resultvarname:The name of the target value 
        #resultvarvalue:The result
        #accuracy:accuracy
        #coverage:coverage
        self.arguments=lst
        self.resultvarname=resultvarname
        self.resultvarvalue=resultvarvalue
        self.accuracy=accuracy
        self.coverage=coverage
    def printrule(self):         #print the rule
        if(len(self.arguments)==1):
            arg=self.arguments[0]
            print("If "+arg.varname+arg.symbol+str(arg.varval)+" then "+self.resultvarname+"="+self.resultvarvalue,end=" ")
        elif(len(self.arguments)>1):
            print("If ",end="")
            for i in range(len(self.arguments)-1):
                arg=self.arguments[i]
                print(arg.varname+arg.symbol+str(arg.varval)+" and ",end="")
            arg=self.arguments[len(self.arguments)-1]
            print(arg.varname+arg.symbol+str(arg.varval)+" then "+self.resultvarname+"="+self.resultvarvalue,end=" ")
        print("with accuracy="+str(self.accuracy)+" and coverage="+str(self.coverage))

In [36]:
def convertintoclass(rules,data_names):
    for Rule in rules:
        rulelst=[]
        for arg in Rule:
            rulelst.append(argument(data_names[arg[0][0]],arg[0][1],arg[0][2]))
        thisrule=rule(rulelst,data_names[-1],Rule[-1][0][3],Rule[-1][1],Rule[-1][2])
        thisrule.printrule()

In [1]:
def findargument(data,possible_moves,accuracy_threshold,coverage_threshold):
    existsargument=False
    totaldata=len(data)
    maxaccuracy=-1         #Max accuracy
    maxcoverage=-1
    bestmove=[-1,-1,-1,-1]
    for moves in possible_moves:
        varindex=moves[0]
        operation=moves[1]
        operationval=moves[2]
        res=moves[3]
        count=0      #Counts the amount of data following the condition(coverage)
        followcount=0#Counts the amount of data following the rule
        for row in data:
            if(operation=="="):
                if(row[varindex]==operationval):
                    count+=1
                    if(row[-1]==res):
                        followcount+=1
            elif(operation==">"):
                if(row[varindex]>operationval):
                    count+=1
                    if(row[-1]==res):
                        followcount+=1
            elif(operation=="<"):
                if(row[varindex]<operationval):
                    count+=1
                    if(row[-1]==res):
                        followcount+=1
        if(count!=0):
            accuracy=followcount/count
            if(count>=coverage_threshold and accuracy>=accuracy_threshold):  #The rules that pass the accuracy and coverage threshold
                if(accuracy>maxaccuracy):   #Go for max accuracy
                    maxaccuracy=accuracy
                    maxcoverage=count
                    bestmove=moves
                elif(accuracy==maxaccuracy and count>maxcoverage):   #Go for better coverage if the accuracy is the same
                    maxaccuracy=accuracy
                    maxcoverage=count
                    bestmove=moves
    return [bestmove,maxaccuracy,maxcoverage]

In [44]:
def findrule(data,possible_moves,accuracy_threshold,coverage_threshold):
    ruleargs=[]
    accuracy=100
    coverage=len(data)
    arg=[[]]
    while arg[0]!=[-1,-1,-1,-1]:
        arg=findargument(data,possible_moves,accuracy_threshold,coverage_threshold)   #Find optimal argument
        if(arg[0]!=[-1,-1,-1,-1]):
            ruleargs.append(arg)
            if(arg[1]==1):
                return ruleargs    #If accuracy is 1, no need to improve
            possible_moves_copy=[]
            data_copy=[]
            for moves in possible_moves:
                if(moves[0]!=arg[0][0]):
                    possible_moves_copy.append(moves)
            for rows in data:
                if(arg[0][1]=="=" and rows[arg[0][0]]==arg[0][2]):
                    data_copy.append(rows)
                if(arg[0][1]==">" and rows[arg[0][0]]>arg[0][2]):
                    data_copy.append(rows)
                if(arg[0][1]=="<" and rows[arg[0][0]]<arg[0][2]):
                    data_copy.append(rows)
            data=data_copy
            possible_moves=possible_moves_copy
    return ruleargs

In [45]:
def prism(data,data_names,accuracy_threshold,coverage_threshold):
    prism_rules=[]
    rule=[]
    first=True
    while rule!=[] or first==True:
        first=False
        possible_outcomes=[]
        possible_moves=[]
        for i in data:
            if(i[-1] not in possible_outcomes):
                possible_outcomes.append(i[-1])
        for j in range(len(data[0])-1):
            for i in range(len(data)):
                val=data[i][j]
                for k in possible_outcomes:
                    if(type(data[i][j]) is int):
                        if ([j,">",data[i][j]] not in possible_moves):
                            possible_moves.append([j,">",data[i][j],k])
                            possible_moves.append([j,"<",data[i][j],k])
                            possible_moves.append([j,"=",data[i][j],k]) #I chose this for integer because it seems the easiest and there is no problem
                    else:
                        if ([j,"=",data[i][j]] not in possible_moves):
                            possible_moves.append([j,"=",data[i][j],k])
        rule=[]
        rule=findrule(data,possible_moves,accuracy_threshold,coverage_threshold)
        if(rule!=[]):
            prism_rules.append(rule)
        data_copy=[]
        for rows in data:
            Pass=True
            for args in rule:
                if(args[0][1]=="="):
                    if(rows[args[0][0]]==args[0][2]):
                        Pass=False
                elif(args[0][1]==">"):
                    if(rows[args[0][0]]>args[0][2]):
                        Pass=False
                elif(args[0][1]=="<"):
                    if(rows[args[0][0]]<args[0][2]):
                        Pass=False
            if Pass is True:
                data_copy.append(rows)
        data=data_copy
    convertintoclass(prism_rules,data_names)

In [46]:
data_file="../data_ml_2020/weather.csv"
import pandas as pd
data=pd.read_csv(data_file)
dataname=data.columns
data_rows = data.to_numpy().tolist()
prism(data_rows,dataname,0.8,2)

If Outlook=Overcast then Play=Yes with accuracy=1.0 and coverage=4
If Temp=Hot then Play=No with accuracy=1.0 and coverage=2
If Humidity=Normal and Windy=False then Play=Yes with accuracy=1.0 and coverage=3


In [47]:
#Test for printrule
Rule=rule([argument("age",">",55),argument("diabetes","=","Yes")],"survive","false",0.8,100)
Rule.printrule()

If age>=55 and diabetes=Yes then survive=false with accuracy=0.8 and coverage=100
